# P2P Protocol Lab: Talk to Yourself, Then to the Network

Welcome to the first hands-on lab for the P2P track. In this notebook you will:

1. **Send messages to your own SQS queue** and read them back (echo pattern)
2. **Build each of the 8 P2P message types** and inspect their JSON structure
3. **Send messages to a classmate's queue** and read their responses
4. **Simulate a mini protocol exchange** (HELLO, PEER_LIST, PING, PONG)

**No long-running processes.** Everything in this notebook is fire-and-read.
You send a message, you immediately read it back. This is how you learn the
protocol before building a real node.

> **Prerequisite:** Read the [P2P Message Protocol Reference](p2p-03-protocol.html)
> before starting this lab. Keep it open as a reference while you work.

> **The Assignment:** After this lab, you will build your own P2P node
> (in any language you want) that interoperates with your classmates' nodes
> over these same SQS queues. This notebook teaches you the protocol
> your node must speak.

---
## Setup

We need the AWS SDK and your credentials. Same as the pub/sub lab.

In [ ]:
# Install the AWS SDK for Python
!pip install -q boto3

In [ ]:
import os

# --- FILL IN YOUR CREDENTIALS ---
os.environ["AWS_ACCESS_KEY_ID"]     = ""  # paste your access key
os.environ["AWS_SECRET_ACCESS_KEY"] = ""  # paste your secret key
os.environ["AWS_DEFAULT_REGION"]    = "us-east-1"

# --- YOUR IDENTITY ---
# Enter your first name in lowercase. This is your node ID.
MY_NAME = ""  # e.g. "hugo", "sam", "phin", "manuel", "bruno"

In [ ]:
import boto3
import json
import uuid
import time
from datetime import datetime, timezone

sqs = boto3.client("sqs", region_name="us-east-1")
print("Connected to AWS SQS.")

### Helpers

Run this cell - it defines the P2P queue URLs and basic send/receive functions.
These are the same SQS queues provisioned for the P2P track (standard queues,
not FIFO - order is NOT guaranteed).

In [ ]:
# ========================================================
# P2P QUEUE CONFIGURATION
# ========================================================

ACCOUNT = "194722398367"
REGION = "us-east-1"
PREFIX = "ds2032"

def queue_url(node_id):
    return f"https://sqs.{REGION}.amazonaws.com/{ACCOUNT}/{PREFIX}-node-{node_id}-p2p"

# All P2P nodes
STUDENTS = ["hugo", "sam", "phin", "manuel", "bruno"]
BOTS = ["bot-alpha", "bot-bravo", "bot-charlie"]
INSTRUCTOR = "gil"
ALL_NODES = STUDENTS + [INSTRUCTOR] + BOTS

# Validate
assert MY_NAME in ALL_NODES, f"Set MY_NAME to one of: {ALL_NODES}"
MY_QUEUE = queue_url(MY_NAME)
print(f"You are: {MY_NAME}")
print(f"Your P2P queue: {MY_QUEUE}")


# --- Send / Receive ---

def send(target_node_id, message_dict):
    """Send a JSON message to a node's P2P queue."""
    body = json.dumps(message_dict)
    resp = sqs.send_message(QueueUrl=queue_url(target_node_id), MessageBody=body)
    return resp

def send_to_self(message_dict):
    """Send a message to YOUR OWN queue (echo pattern)."""
    return send(MY_NAME, message_dict)

def receive(node_id=None, max_messages=10, wait_seconds=3, delete=True):
    """Receive messages from a P2P queue. Defaults to your own."""
    url = queue_url(node_id) if node_id else MY_QUEUE
    resp = sqs.receive_message(
        QueueUrl=url,
        MaxNumberOfMessages=min(max_messages, 10),
        WaitTimeSeconds=wait_seconds,
    )
    results = []
    for m in resp.get("Messages", []):
        body = json.loads(m["Body"])
        results.append(body)
        if delete:
            sqs.delete_message(QueueUrl=url, ReceiptHandle=m["ReceiptHandle"])
    return results

def drain(node_id=None):
    """Remove all messages from a queue. Returns count."""
    count = 0
    while True:
        batch = receive(node_id, max_messages=10, wait_seconds=1, delete=True)
        if not batch:
            break
        count += len(batch)
    return count

def pp(msg):
    """Pretty-print a message."""
    print(json.dumps(msg, indent=2))


print("\nHelpers loaded. Ready to go.")

---
# Section 1: Echo - Talk to Yourself

Before talking to anyone else, let's make sure you can send and receive
on your own queue. This is the foundation of everything.

In P2P, there is no broker relaying messages for you. Every node has its
own SQS queue. When Node A wants to talk to Node B, it sends a message
directly to Node B's queue. Simple as that.

Let's start by talking to yourself.

### 1A: Your first echo

Send a raw JSON message to your own queue and read it back.

In [ ]:
# 1A: Clean start
drained = drain()
if drained:
    print(f"Drained {drained} old messages from your queue.")

# Send a simple message to yourself
send_to_self({"hello": "Is anyone there?", "from": MY_NAME})
print("SENT to your own queue.")

In [ ]:
# 1A: Read it back
time.sleep(1)
msgs = receive()

if msgs:
    print("RECEIVED:")
    pp(msgs[0])
else:
    print("Nothing received. Try running this cell again.")

That's the echo pattern. You sent a message to your own queue, then read
it back. In a real P2P network, you would be sending to *someone else's*
queue - but the mechanics are identical.

Notice: unlike the pub/sub lab, these are **standard** SQS queues (not FIFO).
That means:
- **No guaranteed ordering** - messages may arrive out of order
- **At-least-once delivery** - the same message might arrive twice
- **Best-effort** - but very reliable in practice

Your P2P node will need to handle all of these realities.

---
# Section 2: Building Protocol Messages

The P2P protocol has 8 message types. Every message is a JSON object with
at least these fields:

| Field | Purpose |
|-------|---------|
| `type` | One of: HELLO, PEER_LIST, PING, PONG, VIEW_EVENT, AUDIT_RESULT, CHOKE, UNCHOKE |
| `sender` | Your node ID |
| `timestamp` | When you sent it (ISO 8601 UTC) |
| `msg_id` | A short unique ID |

Let's build each one, send it to yourself, and read it back.

### 2A: Message builder

First, a helper function that creates the common fields.
Every message you build will start with this base.

In [ ]:
# 2A: Base message builder

def base_msg(msg_type):
    """Create a message with the common fields every P2P message needs."""
    return {
        "type": msg_type,
        "sender": MY_NAME,
        "timestamp": datetime.now(timezone.utc).isoformat(),
        "msg_id": uuid.uuid4().hex[:8],
    }

# Test it
sample = base_msg("TEST")
pp(sample)
print(f"\nEvery message you build will start with these 4 fields.")

### 2B: HELLO - Announce yourself

The first message any node sends when it starts up. It tells a bootstrap
node "I exist, here is my queue URL so you can talk to me."

In [ ]:
# 2B: Build and echo a HELLO message

hello = base_msg("HELLO")
hello["queue_url"] = MY_QUEUE

print("BUILT:")
pp(hello)

# Echo it
send_to_self(hello)
time.sleep(1)
echoed = receive()
if echoed:
    print("\nECHOED BACK:")
    pp(echoed[0])
    print(f"\nRoundtrip OK: {echoed[0] == hello}")

The HELLO message carries your `queue_url` so the recipient knows where
to send messages back to you. This is how peer discovery starts.

**Think about it:** In pub/sub, everyone published to a shared SNS topic.
The broker knew all the queues. Here, every node must *manually tell*
other nodes its address. That's the first fundamental difference.

### 2C: PEER_LIST - Share what you know

When you receive a HELLO, you respond with your known peers.
This is the gossip protocol in action.

In [ ]:
# 2C: Build and echo a PEER_LIST message

peer_list = base_msg("PEER_LIST")
peer_list["peers"] = [
    {"node_id": "sam", "queue_url": queue_url("sam")},
    {"node_id": "phin", "queue_url": queue_url("phin")},
]

print("BUILT:")
pp(peer_list)

# Echo
send_to_self(peer_list)
time.sleep(1)
echoed = receive()
if echoed:
    print(f"\nEchoed back with {len(echoed[0].get('peers', []))} peers. Roundtrip OK: {echoed[0] == peer_list}")

### 2D: PING / PONG - Heartbeat

PING checks if a peer is alive. PONG is the response.
The `seq` (sequence number) lets you match which PONG goes with which PING.

In [ ]:
# 2D: Build PING and PONG

ping = base_msg("PING")
ping["seq"] = 1

pong = base_msg("PONG")
pong["seq"] = 1  # echoes the PING's sequence number

print("PING:")
pp(ping)
print("\nPONG:")
pp(pong)

# Echo both
send_to_self(ping)
send_to_self(pong)
time.sleep(1)
echoed = receive(max_messages=2)
print(f"\nEchoed back {len(echoed)} messages:")
for m in echoed:
    print(f"  {m['type']} seq={m['seq']}")

### 2E: VIEW_EVENT - Report a view

The core 2032 data message. Your host reports how many views
a piece of content has received.

In [ ]:
# 2E: Build and echo a VIEW_EVENT

view_event = base_msg("VIEW_EVENT")
view_event["event_id"] = f"evt-{uuid.uuid4().hex[:8]}"
view_event["content_id"] = "show:midnight-run"
view_event["count"] = 150
view_event["ad_id"] = "ad-7"

print("VIEW_EVENT:")
pp(view_event)

send_to_self(view_event)
time.sleep(1)
echoed = receive()
if echoed:
    print(f"\nEchoed: content={echoed[0]['content_id']}, count={echoed[0]['count']}")

### 2F: AUDIT_RESULT - Agree on the truth

After collecting VIEW_EVENTs from peers, a node runs a vote
and publishes the agreed count with a confidence score.

In [ ]:
# 2F: Build and echo an AUDIT_RESULT

audit = base_msg("AUDIT_RESULT")
audit["content_id"] = "show:midnight-run"
audit["agreed_count"] = 150
audit["confidence"] = 0.92
audit["voters"] = [MY_NAME, "sam", "phin", "bot-alpha"]

print("AUDIT_RESULT:")
pp(audit)

send_to_self(audit)
time.sleep(1)
echoed = receive()
if echoed:
    print(f"\nEchoed: agreed={echoed[0]['agreed_count']}, confidence={echoed[0]['confidence']}, voters={len(echoed[0]['voters'])}")

### 2G: CHOKE / UNCHOKE - Enforce reciprocity

CHOKE tells a peer you stopped serving them.
UNCHOKE tells them service resumed.
No extra fields - the message itself is the signal.

In [ ]:
# 2G: Build CHOKE and UNCHOKE

choke = base_msg("CHOKE")
unchoke = base_msg("UNCHOKE")

print("CHOKE:")
pp(choke)
print("\nUNCHOKE:")
pp(unchoke)

# Echo both
send_to_self(choke)
send_to_self(unchoke)
time.sleep(1)
echoed = receive(max_messages=2)
print(f"\nEchoed: {[m['type'] for m in echoed]}")

**You have now built all 8 message types.** Every one of them is just a JSON
dict with common fields plus type-specific data. This is the entire protocol
your P2P node must speak.

| Type | Layer | Extra Fields |
|------|-------|-------------|
| HELLO | Discovery | `queue_url` |
| PEER_LIST | Discovery | `peers` (list of {node_id, queue_url}) |
| PING | Liveness | `seq` |
| PONG | Liveness | `seq` |
| VIEW_EVENT | Application | `event_id`, `content_id`, `count`, `ad_id` |
| AUDIT_RESULT | Application | `content_id`, `agreed_count`, `confidence`, `voters` |
| CHOKE | Incentives | (none) |
| UNCHOKE | Incentives | (none) |

---
# Section 3: Talk to a Classmate

You have been echoing messages to yourself. Now let's send to someone else's queue.

**Important:** Standard SQS queues do not require the recipient to be "online."
Messages will sit in their queue until they read them. This is asynchronous
communication - you send now, they read whenever they are ready.

### 3A: Send a HELLO to a classmate

Pick a classmate's name and send them a HELLO.
They will see it next time they read from their queue.

In [ ]:
# 3A: Send HELLO to a classmate
# Change this to an actual classmate's name!
TARGET = "sam"  # <-- pick someone from: hugo, sam, phin, manuel, bruno

assert TARGET != MY_NAME, "Send to someone else, not yourself!"
assert TARGET in ALL_NODES, f"Unknown node: {TARGET}"

hello = base_msg("HELLO")
hello["queue_url"] = MY_QUEUE

send(TARGET, hello)
print(f"Sent HELLO to {TARGET}!")
print(f"They will see this message next time they check their queue.")
print(f"\nMessage sent:")
pp(hello)

### 3B: Check if anyone said HELLO to you

Read your queue to see if any classmates sent you messages.

In [ ]:
# 3B: Check for incoming messages
msgs = receive(max_messages=10, wait_seconds=5)

if msgs:
    print(f"You have {len(msgs)} message(s):\n")
    for i, m in enumerate(msgs):
        msg_type = m.get("type", "unknown")
        sender = m.get("sender", "?")
        print(f"  [{i+1}] {msg_type} from {sender}")
        if msg_type == "HELLO":
            print(f"       Their queue: {m.get('queue_url', '?')}")
        elif msg_type == "PEER_LIST":
            peers = m.get("peers", [])
            print(f"       Peers shared: {[p['node_id'] for p in peers]}")
        elif msg_type == "PING":
            print(f"       Seq: {m.get('seq')}")
        elif msg_type == "VIEW_EVENT":
            print(f"       Content: {m.get('content_id')}, Count: {m.get('count')}")
else:
    print("No messages. Your classmates haven't sent you anything yet.")
    print("Ask someone to run their cell 3A with your name as TARGET!")

### 3C: Respond with a PEER_LIST

If someone sent you a HELLO, send back your known peers.
This is exactly what a real P2P node does - it's the first step of gossip.

In [ ]:
# 3C: Respond to last HELLO with a PEER_LIST
# If you received a HELLO above, respond to the sender.
# Otherwise, pick a classmate manually.

REPLY_TO = ""  # <-- paste the sender's name from 3B, or pick a classmate

if not REPLY_TO:
    print("Set REPLY_TO to a classmate's name to send them your peer list.")
else:
    peer_list = base_msg("PEER_LIST")
    peer_list["peers"] = [
        {"node_id": MY_NAME, "queue_url": MY_QUEUE},
        # Add any other peers you know about:
        # {"node_id": "phin", "queue_url": queue_url("phin")},
    ]

    send(REPLY_TO, peer_list)
    print(f"Sent PEER_LIST to {REPLY_TO} with {len(peer_list['peers'])} peer(s)")
    pp(peer_list)

---
# Section 4: A Mini Protocol Exchange

Let's simulate a full protocol exchange using your own queue.
This shows you the sequence of messages a real node would handle.

We will simulate what happens when a new node joins the network:
1. New node sends HELLO to a bootstrap node
2. Bootstrap responds with PEER_LIST
3. New node pings all discovered peers
4. Peers respond with PONG
5. New node publishes a VIEW_EVENT
6. An audit collects votes and publishes AUDIT_RESULT

In [ ]:
# 4: Full protocol exchange simulation (all to your own queue)

# Clean start
drain()

print("=== Simulating a node joining the network ===\n")

# Step 1: HELLO
hello = base_msg("HELLO")
hello["queue_url"] = MY_QUEUE
send_to_self(hello)
print(f"1. {MY_NAME} sends HELLO")

# Step 2: PEER_LIST response
peer_list = base_msg("PEER_LIST")
peer_list["sender"] = "bot-alpha"  # simulating the bootstrap's response
peer_list["peers"] = [
    {"node_id": "sam", "queue_url": queue_url("sam")},
    {"node_id": "phin", "queue_url": queue_url("phin")},
    {"node_id": "bot-alpha", "queue_url": queue_url("bot-alpha")},
]
send_to_self(peer_list)
print(f"2. bot-alpha responds with PEER_LIST (3 peers)")

# Step 3: PING to discovered peers
for seq, peer in enumerate(["sam", "phin", "bot-alpha"], start=1):
    ping = base_msg("PING")
    ping["seq"] = seq
    send_to_self(ping)
print(f"3. {MY_NAME} sends PING to 3 peers (seq 1-3)")

# Step 4: PONGs come back
for seq, peer in enumerate(["sam", "phin", "bot-alpha"], start=1):
    pong = base_msg("PONG")
    pong["sender"] = peer
    pong["seq"] = seq
    send_to_self(pong)
print(f"4. All 3 peers respond with PONG")

# Step 5: VIEW_EVENT
view = base_msg("VIEW_EVENT")
view["event_id"] = f"evt-{uuid.uuid4().hex[:8]}"
view["content_id"] = "show:midnight-run"
view["count"] = 150
view["ad_id"] = "ad-7"
send_to_self(view)
print(f"5. {MY_NAME} publishes VIEW_EVENT (count=150)")

# Step 6: AUDIT_RESULT
audit = base_msg("AUDIT_RESULT")
audit["sender"] = "bot-alpha"
audit["content_id"] = "show:midnight-run"
audit["agreed_count"] = 150
audit["confidence"] = 0.92
audit["voters"] = [MY_NAME, "sam", "phin", "bot-alpha"]
send_to_self(audit)
print(f"6. bot-alpha publishes AUDIT_RESULT (agreed=150, conf=0.92)")

print(f"\n=== Sent 9 messages. Reading them back... ===\n")

In [ ]:
# 4: Read back the full exchange
time.sleep(2)

msgs = receive(max_messages=10, wait_seconds=5)

print(f"Received {len(msgs)} messages:\n")
for i, m in enumerate(msgs):
    t = m.get("type", "?")
    s = m.get("sender", "?")

    if t == "HELLO":
        print(f"  [{i+1}] HELLO from {s} (queue={m['queue_url'][-30:]})")
    elif t == "PEER_LIST":
        peers = [p["node_id"] for p in m.get("peers", [])]
        print(f"  [{i+1}] PEER_LIST from {s} -> peers: {peers}")
    elif t == "PING":
        print(f"  [{i+1}] PING from {s} (seq={m['seq']})")
    elif t == "PONG":
        print(f"  [{i+1}] PONG from {s} (seq={m['seq']})")
    elif t == "VIEW_EVENT":
        print(f"  [{i+1}] VIEW_EVENT from {s}: {m['content_id']} count={m['count']}")
    elif t == "AUDIT_RESULT":
        print(f"  [{i+1}] AUDIT_RESULT from {s}: agreed={m['agreed_count']} conf={m['confidence']}")
    else:
        print(f"  [{i+1}] {t} from {s}")

# Check for any remaining
more = receive(max_messages=10, wait_seconds=2)
if more:
    print(f"\n  ... and {len(more)} more messages (standard queues don't guarantee all arrive in one batch)")

Notice anything about the order? With standard SQS queues, messages may
arrive in a **different order** than you sent them. That's by design.
Your P2P node must handle messages in any order.

> **Key insight:** In pub/sub, FIFO queues guaranteed ordering. In P2P,
> we deliberately chose standard queues. Why? Because in a real P2P network,
> messages from different peers naturally arrive out of order anyway.
> There is no central authority imposing a global order.

---
# Section 5: Your Turn

Now that you know the protocol, try these exercises.

### Exercise 1: Build a message dispatcher

Write a function that takes a message dict and prints what type it is
and what the node should do in response.

```python
def dispatch(msg):
    msg_type = msg.get("type")
    if msg_type == "HELLO":
        # what should a node do when it receives a HELLO?
    elif msg_type == "PING":
        # what should happen?
    ...
```

In [ ]:
# Exercise 1: Your dispatcher

def dispatch(msg):
    msg_type = msg.get("type")
    sender = msg.get("sender", "?")

    # TODO: Handle each message type
    # HELLO -> add peer, respond with PEER_LIST
    # PEER_LIST -> merge new peers into your known list
    # PING -> respond with PONG (echo the seq number)
    # PONG -> mark the sender as alive
    # VIEW_EVENT -> record their reported count
    # AUDIT_RESULT -> note the agreed count and confidence
    # CHOKE -> note that sender stopped serving you
    # UNCHOKE -> note that sender resumed serving you

    print(f"Received {msg_type} from {sender}")
    # Your handling code here...

# Test with one of the messages from Section 2
test_msg = base_msg("PING")
test_msg["seq"] = 99
dispatch(test_msg)

### Exercise 2: Peer discovery simulation

Send a HELLO to a bot, then check your queue for their response.
If the bots are running, you should get a PEER_LIST back.

> **Note:** The instructor bots may or may not be running right now.
> If you do not get a response, that is also a valid observation -
> what happens in a P2P network when the bootstrap nodes are down?

In [ ]:
# Exercise 2: Try to discover peers via bot-alpha

# Clean start
drain()

# Send HELLO to bot-alpha
hello = base_msg("HELLO")
hello["queue_url"] = MY_QUEUE
send("bot-alpha", hello)
print(f"Sent HELLO to bot-alpha. Waiting for response...")

# Wait and check for response
time.sleep(10)
responses = receive(max_messages=10, wait_seconds=5)

if responses:
    print(f"\nGot {len(responses)} response(s):")
    for r in responses:
        print(f"  {r.get('type')} from {r.get('sender')}")
        if r.get("type") == "PEER_LIST":
            peers = [p["node_id"] for p in r.get("peers", [])]
            print(f"  Discovered peers: {peers}")
else:
    print("\nNo response from bot-alpha.")
    print("The bot may not be running. In a real P2P network,")
    print("you would try other bootstrap nodes or wait and retry.")

### Exercise 3: Cross-queue conversation

Pair up with a classmate. One of you sends a VIEW_EVENT,
the other reads it from their queue and sends back an AUDIT_RESULT.

Steps:
1. **Person A:** Build a VIEW_EVENT and send it to Person B's queue
2. **Person B:** Read from your queue, inspect the VIEW_EVENT
3. **Person B:** Build an AUDIT_RESULT and send it back to Person A
4. **Person A:** Read your queue and check the audit result

This is the exact exchange that happens in the real 2032 network.

In [ ]:
# Exercise 3A: Send a VIEW_EVENT to your partner

PARTNER = ""  # <-- your partner's node name

if not PARTNER:
    print("Set PARTNER to your partner's name to start.")
else:
    view = base_msg("VIEW_EVENT")
    view["event_id"] = f"evt-{uuid.uuid4().hex[:8]}"
    view["content_id"] = "show:midnight-run"
    view["count"] = 150
    view["ad_id"] = "ad-7"

    send(PARTNER, view)
    print(f"Sent VIEW_EVENT to {PARTNER}!")
    pp(view)

In [ ]:
# Exercise 3B: Check for incoming VIEW_EVENTs and respond with AUDIT_RESULT

msgs = receive(max_messages=10, wait_seconds=5)
view_events = [m for m in msgs if m.get("type") == "VIEW_EVENT"]

if view_events:
    ve = view_events[0]
    print(f"Got VIEW_EVENT from {ve['sender']}:")
    print(f"  Content: {ve['content_id']}, Count: {ve['count']}")

    # Respond with AUDIT_RESULT
    audit = base_msg("AUDIT_RESULT")
    audit["content_id"] = ve["content_id"]
    audit["agreed_count"] = ve["count"]  # We agree (for now)
    audit["confidence"] = 1.0
    audit["voters"] = [MY_NAME]

    send(ve["sender"], audit)
    print(f"\nSent AUDIT_RESULT back to {ve['sender']}: agreed={ve['count']}")
else:
    print("No VIEW_EVENTs yet. Wait for your partner to send one!")

---
# What's Next

You now know the protocol. You can build and send all 8 message types,
and you have seen how they form a conversation between nodes.

**The assignment:** Build your own P2P node that:
1. Runs as a long-lived process on your machine
2. Polls its SQS queue for incoming messages
3. Dispatches each message to the right handler
4. Runs periodic tasks (gossip, heartbeat, choking, reputation)
5. Publishes ViewEvents and participates in audits
6. Interoperates with your classmates' nodes and the instructor bots

**You can write it in any language** - Python, Rust, Go, JavaScript, whatever.
The only contract is the JSON protocol over SQS queues.

> The bots (`bot-alpha`, `bot-bravo`, `bot-charlie`) will be running on
> the instructor's machine. They respond to HELLO, participate in gossip,
> send heartbeats, and publish ViewEvents. Your node should be able to
> discover them, monitor their liveness, and audit their reported counts.

> **Pro tip:** `bot-charlie` has only 50% reporting accuracy.
> Can your reputation system detect that?